## Exercício Desafio

- Digamos que seu chefe pediu para você um relatório da análise dos salários da unidade de San Francisco da empresa. O objetivo dele é entender:

1. Qual foi a evolução do salário médio ao longo dos anos (TotalPay e TotalPayBenefits)
2. Quantos funcionários tivemos ao longo dos anos
3. Qual foi a evolução do total gasto com salário ao longo dos anos (TotalPayBenefits)

- Base de Dados a ser usada: salarios.sqlite

## CREATE

In [ ]:
#cursor.execute("""INSERT INTO nomeDaTabela (Coluna1, Coluna2, etc.) --> INSERT INTO (Comando SQL) + nome da tabela e colunas entre parênteses
#              VALUE
#               ('ValorColuna1, ValorColuna2, etc.')
#               """)

#cursor.commit() --> para perpetuar a alteração do BD.

## READ

In [22]:
#cursor.execute('SELECT * FROM nomeDaTabela')

## UPDATE

## DELETE

### Importação da Base de Dados

In [1]:
import pandas as pd
import pyodbc

In [26]:
dados_conexao = ("""Driver={SQLite3 ODBC Driver};
                    Server=localhost;
                    Database=salarios.sqlite""")

conexao = pyodbc.connect(dados_conexao) 
cursor = conexao.cursor()

cursor.execute('SELECT * FROM salaries')

valores = cursor.fetchall()
descricao = cursor.description

cursor.close()
conexao.close()

In [81]:
colunas = [tupla[0] for tupla in descricao]
salarios_df = pd.DataFrame.from_records(valores, columns=colunas)

display(salarios_df)

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
0,1,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0.00,400184.25,NaN,567595.43,567595.43,2011,,San Francisco,
1,2,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,NaN,538909.28,538909.28,2011,,San Francisco,
2,3,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739.13,106088.18,16452.60,NaN,335279.91,335279.91,2011,,San Francisco,
3,4,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916.00,56120.71,198306.90,NaN,332343.61,332343.61,2011,,San Francisco,
4,5,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.60,9737.00,182234.59,NaN,326373.19,326373.19,2011,,San Francisco,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
148649,148650,Roy I Tillery,Custodian,0.00,0.00,0.00,0.0,0.00,0.00,2014,,San Francisco,PT
148650,148651,Not provided,Not provided,NaN,NaN,NaN,NaN,0.00,0.00,2014,,San Francisco,
148651,148652,Not provided,Not provided,NaN,NaN,NaN,NaN,0.00,0.00,2014,,San Francisco,
148652,148653,Not provided,Not provided,NaN,NaN,NaN,NaN,0.00,0.00,2014,,San Francisco,


### Análise de Dados

In [46]:
# garantindo que estamos só com San Francisco
salarios_df = salarios_df.loc[salarios_df['Agency']=='San Francisco', :]

##### 1. Qual foi a evolução do salário médio ao longo dos anos

In [79]:
tabela_sm = salarios_df.groupby('Year').mean()
display(tabela_sm[['TotalPay', 'TotalPayBenefits']])

,TotalPay,TotalPayBenefits
Year,,
2011,71744.103871,71744.103871
2012,74113.262265,100553.229232
2013,77611.443142,101440.519714
2014,75463.918140,100250.918884


##### 2. Quantos funcionários tivemos ao longo dos anos

In [84]:
tabela_qtde = salarios_df.groupby('Year').count()
tabela_qtde = tabela_qtde[['Id']]
tabela_qtde = tabela_qtde.rename(columns={'Id': 'Qtde.:'})
display(tabela_qtde)

,Qtde.:
Year,
2011,36159
2012,36766
2013,37606
2014,38123


##### 3. Qual foi a evolução do total gasto com salário ao longo dos anos

In [88]:
def formatar(valor):
    return 'R${:,.2f}'.format(valor)

tabelaGasto = salarios_df.groupby('Year').sum()
tabelaGasto = tabelaGasto[['TotalPay', 'TotalPayBenefits']]
tabelaGasto['TotalPay'] = tabelaGasto['TotalPay'].apply(formatar)
tabelaGasto['TotalPayBenefits'] = tabelaGasto['TotalPayBenefits'].apply(formatar)
display(tabelaGasto)

,TotalPay,TotalPayBenefits
Year,,
2011,"R$2,594,195,051.88","R$2,594,195,051.88"
2012,"R$2,724,848,200.44","R$3,696,940,025.96"
2013,"R$2,918,655,930.80","R$3,814,772,184.37"
2014,"R$2,876,910,951.26","R$3,821,865,780.60"
